In [ ]:
from dbfread import DBF, DBFNotFound
import os
from datetime import date
import pandas as pd

In [ ]:
herkunft = {
    "1": "Schriftlich",
    "2": "Fax",
    "3": "Telefon",
    "4": "Internet",
    "5": "Call-Center",
    "6": "Ladenverkauf",
    "7": "Vertreter",
    "8": "E-Mail",
    "9": "Anrufbeantworter/Mailbox",
    "B": "Beleglesung",
    "E": "Marktplätze",
    "F": "Amazon-Fulfillment",
    "M": "Messe",
    "S": "SMS",
    "nan": "Ohne/Unbekannt",
}

reasons = {
    "1": "Lagerstorno",
    "10": "Auftrag in Angebot",
    "11": "Artikel ausverkauft",
    "12": "offener Saldo",
    "13": "Weihnachten zu spät",
    "14": "Terminauftrag",
    "15": "Auftrag doppelt",
    "16": "TK Ausland",
    "17": "Fehler bei Erfassung",
    "18": "Kundenwunsch/Kundenstorno",
    "19": "Aufträge zusammengefasst",
    "2": "Kunde lehnt Vorauskasse ab",
    "20": "Zahlung abgelehnt",
    "3": "Vorkasse nicht erfolgt",
    "4": "Unzufriedenheit Kunde",
    "5": "Empfänger unbekannt",
    "6": "Dubios",
    "7": "sonstiges",
    "8": "Testauftrag",
    "9": "Lieferzeit zu lang",
}

In [ ]:
def read_dbf(file_path):
    table = DBF(file_path, load=True, encoding="cp850", ignore_missing_memofile=True)
    df = pd.DataFrame(iter(table))
    return df


def write_csv(df, file_name):
    df.to_csv(file_name, sep=";", encoding="cp850", index=False)

In [ ]:
this_month = date.today().strftime("%Y%m")
last_month = f"{int(this_month) - 1}"


In [ ]:
auf_gel = []
lands = ["F01", "F02", "F03", "F04"]
# FILE_NAME = 'V4AUProKopf202501'
this_month = f"V4AUProKopf{this_month}"
file = f"V4AUProKopf{last_month}"

for LAND in lands:
    # dbf_file_path = f'/Volumes/DATA/{LAND}/{FILE_NAME}.dbf'
    dbf_file_path = f"/Volumes/DuG/VS/Dg/VC2/{LAND}/AUFTRAG/{file}.dbf"

    csv_file_path = f"/Volumes/MARAL/CSV/{LAND}/{file}.csv"

    if os.path.exists(csv_file_path):
        print(f"The file at {csv_file_path} exists.")
        modification_time = os.path.getmtime(csv_file_path)
        modification_date = date.fromtimestamp(modification_time)
        print(modification_date)

        today_date = date.today()
        print(today_date)
        if modification_date != today_date:
            dbf = read_dbf(dbf_file_path)
            write_csv(dbf, csv_file_path)

        df = pd.read_csv(csv_file_path, encoding="cp850", on_bad_lines="skip", sep=";")
    else:
        print(f"The file at {csv_file_path} does not exist.")
        dbf = read_dbf(dbf_file_path)
        write_csv(dbf, csv_file_path)
        df = pd.DataFrame(iter(dbf))
    auf_gel.append(dbf)

dbf_ges = pd.concat(auf_gel)

In [ ]:
nf_gel = []
lands = ["F01", "F02", "F03", "F04"]
# FILE_NAME = 'V4AUProKopf202501'
file = "V2SC1010"


for LAND in lands:
    # dbf_file_path = f'/Volumes/DATA/{LAND}/{FILE_NAME}.dbf'
    dbf_file_path = f"/Volumes/DuG/VS/Dg/VC2/{LAND}/SCHNITT/{file}.dbf"

    csv_file_path = f"/Volumes/MARAL/CSV/{LAND}/{file}.csv"

    if os.path.exists(csv_file_path):
        print(f"The file at {csv_file_path} exists.")
        modification_time = os.path.getmtime(csv_file_path)
        modification_date = date.fromtimestamp(modification_time)
        print(modification_date)

        today_date = date.today()
        print(today_date)
        if modification_date != today_date:
            dbf_nf = read_dbf(dbf_file_path)
            write_csv(dbf_nf, csv_file_path)

        df_nf = pd.read_csv(
            csv_file_path, encoding="cp850", on_bad_lines="skip", sep=";"
        )
    else:
        print(f"The file at {csv_file_path} does not exist.")
        dbf_nf = read_dbf(dbf_file_path)
        write_csv(dbf_nf, csv_file_path)
        df_nf = pd.DataFrame(iter(dbf_nf))
    nf_gel.append(dbf_nf)

nf_ges = pd.concat(nf_gel)

In [ ]:
nf_ges_copy = nf_ges.copy()
dbf_ges_copy = dbf_ges.copy()

In [ ]:
nf_ges_copy = nf_ges.drop_duplicates()

In [ ]:
dbf_ges_copy = dbf_ges_copy[
    [
        "ADR_NR",
        "AUFTRAG_NR",
        "DATUM",
        "BEST_WERT",
        "RECH_NR",
        "SYS_ANLAGE",
        "LAGERPLATZ",
        "ZAHLART",
        "ERFASSER",
        "LKZ",
    ]
]
dbf_ges_copy = dbf_ges_copy.rename(
    columns={"LAGERPLATZ": "LOESCHGRUND", "ERFASSER": "BEARBEITER"}
)
dbf_ges_copy["AUFTRAG_NR"] = (
    dbf_ges_copy["AUFTRAG_NR"].astype(str).str.replace(".0", "").str.zfill(10)
)

In [ ]:
nf_ges_copy = nf_ges_copy[
    [
        "AUFTRAG_NR",
        "DATUM",
        "MEDIACODE",
        "HERKUNFT",
        "RECH_ART",
        "ART_NR",
        "GROESSE",
        "FARBE",
        "MENGE",
        "PREIS",
        "MWST",
        "MWST_KZ",
        "RETOUREGRD",
        "RETOUREART",
        "EK",
        "WARENGR",
    ]
]
nf_ges_copy.sort_values(by="DATUM", ascending=True)

In [ ]:
nf_ges_copy = nf_ges_copy[nf_ges_copy["NACHFRAGE_DATUM"] >= "2024-01-01"]
nf_ges_copy["AUFTRAG_NR"] = (
    nf_ges_copy["AUFTRAG_NR"].astype(str).str.replace(".0", "").str.zfill(10)
)

df_merge = dbf_ges_copy.merge(nf_ges_copy, on="AUFTRAG_NR", how="left")
df_merge.to_excel("test.xlsx", index=False)
